In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import efficientnet as efn
from common.model_utils import input_size
import os

2024-03-17 19:53:08.472623: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Function to load TFLite model
def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

# Function to preprocess image
def preprocess_image(image_path, img_size):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_size, img_size))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [3]:
data_folder = ".../dev/Training Data"
landmark_data_folder = ".../dev/Training Landmarks Data"
result_folder = ".../dev/Result"
landmark_result_folder = ".../dev/Landmark Result"
models_folder = ".../dev/Benchmark Models"
landmark_models_folder = ".../dev/Landmark Models"
tuning_folder = ".../dev/Tuning"
landmark_tuning_folder = ".../dev/Landmark Tuning"

In [4]:
model_path = "tflite/BaseB0.tflite"
LANDMARK = False

if LANDMARK:
    tuning_dir = landmark_tuning_folder
    data_dir = landmark_data_folder
    model_dir = landmark_models_folder
    result_dir = landmark_result_folder
else:
    tuning_dir = tuning_folder
    data_dir = data_folder
    model_dir = models_folder
    result_dir = result_folder

test_path = os.path.join(".", data_dir, "Test")
model = model_dir = "tflite"
model_path = os.path.join(model_dir, f"BaseB0.tflite")
model = load_tflite_model(model_path)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [6]:
# test_img_gen = ImageDataGenerator(
#     preprocessing_function=efn.preprocess_input
# )


# test_generator = test_img_gen.flow_from_directory(
#     directory=test_path,
#     target_size=(input_details[0]['shape'][1], input_details[0]['shape'][2]),
#     batch_size=1,
#     class_mode=None,
#     shuffle=False,
#     seed=123,
# )

In [7]:
# # Function to perform inference on an image batch
# def predict_batch(image_batch):
#     interpreter.set_tensor(input_details[0]['index'], image_batch)
#     interpreter.invoke()
#     output_data = interpreter.get_tensor(output_details[0]['index'])
#     return output_data

# # Initialize variables for accuracy calculation
# total_images = test_generator.samples
# correct_predictions = 0

# # Iterate over test data generator and perform inference
# for i in range(total_images):
#     image_batch = test_generator.next()
#     predicted_scores = predict_batch(image_batch)
#     predicted_classes = np.argmax(predicted_scores, axis=1)
#     # You can calculate accuracy here if you have ground truth labels
#     # Otherwise, you can print predictions for further analysis

In [8]:
# Test the model
test_img_gen = ImageDataGenerator(preprocessing_function=None)  # No preprocessing for TFLite models

img_size = (input_details[0]['shape'][1], input_details[0]['shape'][2])  # Define your function to get the input size based on the model type

test_generator = test_img_gen.flow_from_directory(
    directory=test_path,
    target_size=img_size,
    batch_size=1,  # Set batch size to 1 for TFLite inference
    class_mode=None,
    shuffle=False,
    seed=123,
)

# Perform inference and evaluate model
predicted_class_indices = []
for i in range(len(test_generator)):
    image_batch = test_generator.next()
    model.set_tensor(model.get_input_details()[0]['index'], image_batch)
    model.invoke()
    output_data = model.get_tensor(model.get_output_details()[0]['index'])
    predicted_class_indices.extend(np.argmax(output_data, axis=1))

# Get ground truth labels
y = test_generator.classes

# Calculate evaluation metrics
accuracy = metrics.accuracy_score(y, predicted_class_indices)
precision = metrics.precision_score(y, predicted_class_indices, average="binary")
recall = metrics.recall_score(y, predicted_class_indices, average="binary")
auc = metrics.roc_auc_score(y, predicted_class_indices)

print("Model Accuracy:", accuracy)
print("Model Precision:", precision)
print("Model Recall:", recall)
print("AUC:", auc)

# Save results to CSV
result_dir = "/home/FYP/dion0020/deepfake-detector/dev/tflite/results"
filenames = [os.path.split(i)[1] for i in test_generator.filenames]
actual_labels = [os.path.split(i)[0] for i in test_generator.filenames]
actual_labels = ["0" if label == "manipulated" else "1" for label in actual_labels]

results_df = pd.DataFrame({
    "id": filenames,
    "actual label": actual_labels,
    "pred label": predicted_class_indices
})

os.makedirs(result_dir, exist_ok=True)
results_df.to_csv(os.path.join(result_dir, model_type + "_out.csv"))

# Save model results to JSON
models_result = {
    model_type: {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "AUC": auc
    }
}

with open(os.path.join(result_dir, "model_results.json"), "w") as outfile:
    json.dump(models_result, outfile)

# Save model prediction results to CSV
model_prediction_results = pd.DataFrame.from_dict(models_result, orient="index")
model_prediction_results.to_csv(os.path.join(result_dir, "prediction_results.csv"))

Found 42562 images belonging to 2 classes.
